In [1]:
import cv2
import numpy as np
import time

#start reading webcam
cap = cv2.VideoCapture(0)

#read initial frames
ret, frame1 = cap.read()
ret, frame2 = cap.read()

#get screen dimensions
frame_height, frame_width = frame1.shape[:2]

#define center tolerance (I will need to test and tweak this)
center_tolerance = 50

while cap.isOpened():
    #compute the absolute difference between frames
    diff = cv2.absdiff(frame1, frame2)
    
    #convert the difference to grayscale
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    #apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    #get threshold
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    
    #dilate our threshold
    dilated = cv2.dilate(thresh, None, iterations=3)
    
    #get contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Convert frame to HSV color space
    hsv = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)

    # Define range of blue color in HSV
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame1,frame1, mask= mask)

    # Convert the result to grayscale and apply thresholding
    gray1 = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    _, thresh1 = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

    # Find contours
    contours1, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #draw contours to detect motion
    for contour in contours:
        if cv2.contourArea(contour) < 700:
            continue
        
        #get bounding box coordinates for each contour
        (x, y, w, h) = cv2.boundingRect(contour)
        
        #mark object with a rectangle
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        #calculate center of bounding box
        center_x = x + w // 2
        center_y = y + h // 2
        
        #check if the object is (roughly) in the center of the screen
        screen_center_x = frame_width // 2
        screen_center_y = frame_height // 2
        
        if (abs(center_x - screen_center_x) <= center_tolerance and
            abs(center_y - screen_center_y) <= center_tolerance):
            #object is in the center of the screen, take a screenshot
            screenshot_filename = f'screenshot_{int(time.time())}.png'
            cv2.imwrite(screenshot_filename, res)
            print(f"Screenshot taken: {screenshot_filename}")
    
    #display resulting frame
    cv2.imshow("Motion Detection", res)
    
    #update frames
    frame1 = frame2
    ret, frame2 = cap.read()

    #exit loop
    if cv2.waitKey(10) == ord('q'):
        break

#close up program
cap.release()
cv2.destroyAllWindows()


Screenshot taken: screenshot_1727397208.png
Screenshot taken: screenshot_1727397213.png
Screenshot taken: screenshot_1727397213.png
Screenshot taken: screenshot_1727397214.png
